In [ ]:
import pandas as pd
import numpy as np
import random
import math
import gc
from simpledbf import Dbf5
from pandas import DataFrame

### Open dbf files and convert to dataframes for processing

In [ ]:
Student_DBF = Dbf5('c:\DATA\Reduc2.dbf')
School_DBF = Dbf5('c:\DATA\RNEW1.dbf')
#Student_DBF = Dbf5('c:\DATA\SMA0KM.dbf')

In [ ]:
School_frame = School_DBF.to_dataframe()
Student_frame = Student_DBF.to_dataframe()

In [ ]:
grouped_school = School_frame.groupby('CSOED_1')

### Define Functions

In [ ]:
def YvsS(Y_Solution, N_Solution):
    global Added_Records
    Added_Records = False
    Y_Small_Area = float(Y_Solution['Small_Area'])
    Y_Details = float(Y_Solution['Details'])
    N_Small_Area = float(N_Solution['Small_Area'])
    N_Details = float(N_Solution['Details'])
    
    if (Y_Small_Area >= N_Small_Area or Y_Details <= N_Details):
        return True

In [ ]:
def UpdateM(N_Solutions, temp_M_Solutions):
    
    for index, row in N_Solutions.iterrows():
        Insert = True
        if len(temp_M_Solutions.index) == 0:
            temp_M_Solutions = temp_M_Solutions.append(N_Solutions.loc[N_Solutions['Counter'] == row['Counter']])
        else:
            for index2, row2 in temp_M_Solutions.iterrows():
                if (row['Small_Area'] >= row2['Small_Area'] and row['Details'] <= row2['Details']): 
                    temp_M_Solutions = temp_M_Solutions[temp_M_Solutions.Counter != row2['Counter']]
            for index2, row2 in temp_M_Solutions.iterrows(): 
                if (row['Small_Area'] <= row2['Small_Area'] and row['Details'] >= row2['Details']):
                    Insert = False
            if Insert: 
                temp_M_Solutions = temp_M_Solutions.append(N_Solutions.loc[N_Solutions['Counter'] == row['Counter']])
                Added_Records = True
    return temp_M_Solutions
    

In [ ]:
def swap_one():
    
    Modified_Random_Selection = pd.DataFrame()
    merged = EDSelected_Students.merge(Y_Selection, how='left', indicator=True)
    EDAvailable_Students = merged[merged['_merge']=='left_only']
    EDAvailable_Students = EDAvailable_Students.drop('_merge', 1)
    groups = Y_Selection.groupby('ROLL_NO')
    for ROLL_NO, group in groups:
        Available_Students = EDAvailable_Students.loc[EDAvailable_Students['ROLL_NO'] == ROLL_NO]
        Selection = Y_Selection.loc[Y_Selection['ROLL_NO'] == ROLL_NO]
        drop_index = np.random.choice(Selection.index, 1, replace=False)
        if len(Available_Students.index) > 0:
            Random_Selection = Selection.drop(drop_index)
            New1 = Available_Students.sample(n=1)
            Random_Selection = Random_Selection.append(New1)
            Modified_Random_Selection = Modified_Random_Selection.append(Random_Selection)
        else:
            Modified_Random_Selection = Modified_Random_Selection.append(Selection)
    return Modified_Random_Selection

In [ ]:
def statistics(EDRandom_Selection):
    d = {'CSOED': [CSOED_1]}
    Solution = pd.DataFrame(d)
    Lone_Parent = abs(Act_PARENT_LON - (EDRandom_Selection['PARENT_LON'].sum()/Ed_ENROLLED))
    Rented = abs(Act_RENTED - (EDRandom_Selection['RENTED'].sum()/Ed_ENROLLED))
    Unemployed = abs(Act_UNEMPLOYED - (EDRandom_Selection['UNEMPLOYED'].sum()/Ed_ENROLLED))
    Tertiary = abs(Act_TERTIARY_P - (EDRandom_Selection['TERTIARY_P'].sum()/Ed_ENROLLED))
    Solution['Small_Area'] = (EDRandom_Selection['SMALL_AREA'].sum()/Ed_ENROLLED)
    Solution['Details'] = ((Lone_Parent+Rented+Unemployed+Tertiary)/4)
    return Solution    
        

In [ ]:
def accept_decision(probability):
    return random.random() < probability

### Main Body

In [ ]:
for CSOED_1, group in grouped_school:
    n = 0
    r = 10
    Alpha = 1.05
    Single_Selection = pd.DataFrame()
    Total_Random_Selections = pd.DataFrame()
    Overall_Solution = pd.DataFrame()
    Overall_Solution_Records = pd.DataFrame()
    M_Random_Selections = pd.DataFrame()
    M_Solutions = pd.DataFrame()
    N_Random_Selections = pd.DataFrame()
    N_Solutions = pd.DataFrame()
    Selected_Students = pd.DataFrame()
    print("declarations")
    for _ in range(r):
        EDSelected_Students = pd.DataFrame()
        S_Random_Selection = pd.DataFrame() 
        Ed_ENROLLED = 0
        Selected_Schools = School_frame.loc[School_frame['CSOED_1'] == CSOED_1]
        for index, row in Selected_Schools.iterrows():
                #SETUP
            Act_PARENT_LON = row['LONE_PAREN']
            Act_RENTED = row['RENTED']
            Act_UNEMPLOYED = row['NO_WORKERS']
            Act_TERTIARY_P = row['TERTIARY_P']
            ROLL_NO = row['ROLL_NO']
            TOTAL_ENROLLED = int(row['FSL_TOTAL'])
                #Random Selection
            Selected_Students = Student_frame.loc[Student_frame['ROLL_NO'] == ROLL_NO]
            EDSelected_Students = EDSelected_Students.append(Selected_Students)
            print(len(Selected_Students.index))
            if len(Selected_Students.index) < TOTAL_ENROLLED:
                Random_Selection = Selected_Students.sample(n=TOTAL_ENROLLED,replace=True)
            else:
                Random_Selection = Selected_Students.sample(n=TOTAL_ENROLLED,replace=False)
            S_Random_Selection = S_Random_Selection.append(Random_Selection)
            Ed_ENROLLED = (Ed_ENROLLED + TOTAL_ENROLLED)
        #print("SChool records gen")
        S_Solution = statistics(S_Random_Selection)
        #S_Solution = statistics(S_Random_Selection, Ed_ENROLLED,School_frame,CSOED_1)
        S_Solution['Counter'] = n
        N_Solutions = N_Solutions.append(S_Solution, ignore_index=True)
        S_Random_Selection['Counter'] = n
        N_Random_Selections = N_Random_Selections.append(S_Random_Selection)
        n=n+1
    #print("ED records gen")
            #Solution Stats for this run
    Total_Random_Selections = Total_Random_Selections.append(N_Random_Selections)
        #with open('Selections.csv', 'a') as f:
        #    Total_Random_Selections.to_csv(f, header=False)
    M_Solutions = UpdateM(N_Solutions, M_Solutions)
    if len(Selected_Students.index) == TOTAL_ENROLLED:
        print("ED finished early")
    else:
        #for _ in range(r):
        for index, row in N_Solutions.iterrows():
            CurrentN = pd.DataFrame()
            T = 1
            Lambda1 = 0.5
            Lambda2 = 0.5
            Count = int(row['Counter'])
            #print("Solution", Count)
            Y_Selection = Total_Random_Selections.loc[Total_Random_Selections['Counter'] == Count]
            while (T > 0.3):
                #print("Temp", T)
                Y_Selection = swap_one()
                Y_Selection['Counter'] = n
                Y_Solution = statistics(Y_Selection)
                Y_Solution['Counter'] = n
                Total_Random_Selections = Total_Random_Selections.append(Y_Selection)
                n = n+1
                CurrentN = N_Solutions.loc[N_Solutions['Counter'] == Count]
                if YvsS(Y_Solution, CurrentN):
                    M_Solutions = UpdateM(Y_Solution, M_Solutions)
                N_Solutions['test'] = (abs(N_Solutions['Small_Area'] - row['Small_Area']) + abs(N_Solutions['Details'] - row['Details']))
                m = min(i for i in N_Solutions['test'] if i > 0)
                Nearest_Neighbour = N_Solutions.loc[N_Solutions['test'] == m]
                N_Solutions = N_Solutions.drop('test',1)
                S_Small_Area = float(row['Small_Area'])
                v= min(Nearest_Neighbour.index)
                N_Small_Area = float(Nearest_Neighbour.loc[v]['Small_Area'])
                S_Details = float(row['Details'])
                N_Details = float(Nearest_Neighbour.loc[v]['Small_Area'])
                if S_Small_Area >= N_Small_Area:
                    Lambda1 = Lambda1*Alpha
                else:
                    Lambda1 = Lambda1/Alpha
                if S_Details <= N_Details:
                    Lambda2 = Lambda2*Alpha
                else:
                    Lambda2 = Lambda2/Alpha
                Lambda1Norm = (Lambda1/Lambda1+Lambda2)
                Lambda2Norm = (Lambda2/Lambda1+Lambda2)
                Prob = math.exp(((Lambda1Norm*(S_Small_Area-N_Small_Area)) + (Lambda2Norm*(S_Details-N_Details)))/T)
                Prob = min(1,Prob)
                #print(Prob)
                decision = accept_decision(Prob)
                #print(decision)
                if decision == True:
                    N_Solutions.loc[index] = Y_Solution.loc[0]
                    Y_Selection = Total_Random_Selections.loc[Total_Random_Selections['Counter'] == (n-1)]
                    Count = n-1
                T = T-0.02
        #FInalise and pick one
        Final_Random_Selections = Total_Random_Selections.loc[Total_Random_Selections['Counter'].isin(M_Solutions['Counter'])]
        Total_Random_Selections= Total_Random_Selections.loc[Total_Random_Selections['Counter'].isin(N_Solutions['Counter'])]
            #Total_Random_Selections= Total_Random_Selections.append(N_Random_Selections)
            #gc.collect()
        with open('selections.csv', 'a') as f:
            Final_Random_Selections.to_csv(f, header=False)
        with open('results.csv', 'a') as g:
            M_Solutions.to_csv(g, header=False)
        del M_Solutions
        gc.collect()
        print(CSOED_1, " Finished")  
print("Finished")    

### Splitting FIle into smaller catchement 

In [ ]:
for CSOED_1, group in grouped_school:
    Selection = pd.DataFrame()
    Students = pd.DataFrame()
    TOTAL_ENROLLED = 0
    Selected_Schools = School_frame.loc[School_frame['CSOED_1'] == CSOED_1]
    for index, row in Selected_Schools.iterrows():
        ROLL_NO = row['ROLL_NO']
        S = Student_frame.loc[Student_frame['ROLL_NO'] == ROLL_NO]
        Students = Students.append(S)
        TOTAL_ENROLLED = TOTAL_ENROLLED + int(row['FSL_TOTAL'])
        #print(TOTAL_ENROLLED)
    Selection = Students.loc[Students['SMALL_AREA'] == 1]
    Available = Students.loc[Students['SMALL_AREA'] == 0]
    #Sampsize = int(abs(TOTAL_ENROLLED - len(Selection.index))*1.5)
    #print("SEL",len(Selection.index))
    TOTAL_ENROLLED = int(TOTAL_ENROLLED*0.4)
    if len(Available.index) > 0:
        if len(Available.index) < TOTAL_ENROLLED:
            Random_Selection = Available.sample(n=TOTAL_ENROLLED,replace=True)
        else:
            Random_Selection = Available.sample(n=TOTAL_ENROLLED,replace=False)
        Selection = Selection.append(Random_Selection)
    #print("SELALL",len(Selection.index))
    with open('reducedcatchment.csv', 'a') as f:
        Selection.to_csv(f, header=False)
    print(CSOED_1)

print("Finished")    

### SELECT FINAL RESULTS RANDOMLY

In [ ]:
All = pd.DataFrame()
Select_DBF = Dbf5('c:\DATA\Results.dbf')
Select_frame = Select_DBF.to_dataframe()
grouped_Select = Select_frame.groupby('CSOED')
for CSOED, group in grouped_Select:
    Selected = Select_frame.loc[Select_frame['CSOED'] == CSOED]
    New1 = Selected.sample(n=1)
    All = All.append(New1)
All.to_csv('FinalSelection.csv', header=True)